## Columns overall

Overall, data give a full picture of the company's entire customer base as of 2015.
This is to say that we are able to observe all clients with an active contract in 2015 end their complete contracts history until 2020.

------------------------- **<i>CLIENT</i>** ---------------------------------------------

Dataset containing anagraphical information on the clients as of december 2020

COD_TIPO_PERSONA: Client Type

ANZIANITA_ATTIVA: active seniority of the client

NUMERO_SINISTRI: overall number of accidents

PROV_RES: province of residence

RATING: risk rating

COD_GEND_PF: gender

FIGLI: Flag for children in the household

FLG_APP: Flag for use of the mobile application

SUM_IMP_PREMIO_CTTO: sum of premia paid by the client to the insurance company

ID_CLIENTE: identification number of the client

ANNO_NASC_PF: year of birth of the client


------------------------- **<i>CONTRACT</i>** ---------------------------------------------

Dataset containing information on the contracts with the perimeter as of december 2020

ID_CLIENTE: identification number of the client

CANALE_PROV: channel of purchase

RISCHIO: risk type

PRODOTTO: product type

STATO: status of the contract

SIT ASSUN: underwriting condition

NUM_RATE: number of installments

MODALITA_PAGAMENTO_STIPULA: payment modality at underwriting

MODALITA_PAGAMENTO_RATE: payment modality at installments

RAGG_BUSINESS: business grouping

AMT_PREMIO_RCA: premium amount of the RCA component (RCA: Responsabilità Civile Autoveicoli)

SCONTO_TOT_RCA: overall discount on the RCA component

AMT_PREMIO_CTTO: premium amount of the contract

AMT_SCONTO_CTTO: discount amount of the contract

GAR_[...]: set of warranty indicators

CONTRACT_CODE: key identifier of the contract

------------------------- **<i>CONTRACT_DATES</i>** ---------------------------------------------

Dataset containing information on relevant dates of the contracts

DAT_DECORRENZA: effective date (date since when the contract is effective)

DAT_CHIUSURA: closing date (when the contract has been closed by the client)

DAT_SCADENZA: expiration date (natural end of the contract)


------------------------- **<i>INFOMOTOR</i>** ---------------------------------------------


Dataset containing information on relevant characteristics of the insured good (vehicle)

COD_TIPO_VEICOLO: vehicle type

DATA_PRIMA_IMMATRICOLAZIONE: first matriculation date

IMPORTO_VALORE_COMMERCIALE: commercial value

COD_TIPOLOGIA_GUIDATORI: drive type

KM_ANNUI_PREVISTI: yearly expected km

POTENZA_KW: kw power

COD_TIPO_ALIMENTAZIONE: power supply

DESC_MARCA: vehicle brand

DESC_TIPO_VEICOLO: vehicle type

TIPO_CARROZZERIA: car body

DESC_TIPO_CARROZZERIA: car body description 

PRESENZA_ABS: presence of ABS

PRESENZA_AIRBAG: presence of Airbag

PRESENZA_ANTIF: presence of theft protection system

CONTRACT_CODE: code of the contract



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os

# Preprocessing
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler


# models
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf


In [2]:
# Creating relevant variables
rs = 42 #Random_state
np.random.seed(rs)
cv = 5
%matplotlib inline
plt.rcParams['figure.figsize'] = [40, 40]

In [3]:
# Relevant Functions

# Function to detect and print outliers 
def detect_outliers(data):
    # mean, standard deviation and 3-sigma of the data
    mean = np.mean(data)
    std = np.std(data)
    threesigma = 3 * std
    
    # print upper, lower boundary and boxplot
    sns.boxplot(data, orient="v")
    plt.show()
    lower, upper = mean-3*std, mean+3*std
    print(f"Upper and lower boundary is: {lower}/{upper}")
    
    # identify outliers and return the outliers
    outliers = [x for x in data if np.abs(x - mean) > threesigma]
    print(f"There are {len(outliers)} outliers based on three-sigma rule")

# Function to delete the outliers 
def delete_outliers(data, df):
    # detecting and dropping outliers 
    original_shape = df.shape
    mean = np.mean(data)
    std = np.std(data)
    outliers = np.abs(data-mean) > (3*std)
    outliers_num = len(df[outliers])
    df.drop(index=data[outliers].index, inplace=True)
    
    # print what was deleted
    print("Number of outliers deleted:", outliers_num)
    print ("Shape of dataframe with Ouliers: ",original_shape)
    print ("Shape of Dataframe After Deleting the Ouliers: ",df.shape)

# Function to plot the Correlation Heatmap
def correlation_heatmap(corr):
    mask = np.array(corr)
    mask[np.tril_indices_from(mask)] = False
    
    # plot the correlation
    fig,ax= plt.subplots()
    fig.set_size_inches(10,10)
    sns.heatmap(corr, mask=mask,vmax=.8, square=True,annot=True)

# Function to calculate RMSLE - Root Mean Squared Logarithmic Error   
def rmsle(y_true, y_pred):
    """
    Custom RMSLE scorer function.
    """
    log_diff = np.log1p(y_pred) - np.log1p(y_true)
    rmsle_score = np.sqrt(np.mean(log_diff**2))
    return rmsle_score

In [4]:
#Read the tables

df = pd.read_csv("CHURN_AUTO_CONTRACT_DATES.csv")
df_sorted = df.sort_values('ID_CLIENTE')

In [5]:


#na_status(df_sorted)

dfClient = pd.read_csv("CHURN_CLIENT_INFO.csv")
#na_status(dfClient)

dfContrAuto = pd.read_csv("CHURN_CONTRACT_AUTO.csv")
#na_status(dfContrAuto)

dfInfoAuto = pd.read_csv(("CHURN_INFOAUTO.csv"), encoding='ISO-8859-1')
#na_status(dfInfoAuto)




C:\Users\joaov\AppData\Local\Temp\ipykernel_2244\3305856234.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  dfClient = pd.read_csv("CHURN_CLIENT_INFO.csv")


In [6]:
df_new=df_sorted[df_sorted["DAT_DECORRENZA"]!=df_sorted["DAT_CHIUSURA"]]
#df_new.drop(columns=["GAR_RCRUOTE","GAR_TELEMATICA_APP_E_GO","MODALITA_PAGAMENTO_RATE","GAR_TUTELA_AGGRESSIONI"],inplace=True,axis=1)
# Identifica le colonne che iniziano con "GAR_" e hanno più del XX% di valori nulli
tabelle_da_elimare = [colonna for colonna in df_new.columns if df_new[colonna].isnull().mean() > 0.1]

df_new = df_new.drop(tabelle_da_elimare, axis=1)


In [7]:
#Transforming our date columns into a datetime type column:
df_new['DAT_DECORRENZA'] = [datetime.strptime(x, "%d%b%Y:%H:%M:%S") for x in df_new['DAT_DECORRENZA']]
df_new['DAT_SCADENZA'] = [datetime.strptime(x, "%d%b%Y:%H:%M:%S") for x in df_new['DAT_SCADENZA']]
df_new['DAT_CHIUSURA'] = [datetime.strptime(x, "%d%b%Y:%H:%M:%S") for x in df_new['DAT_CHIUSURA']]
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6748575 entries, 1571600 to 1581358
Data columns (total 29 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   CONTRACT_CODE               object        
 1   ID_CLIENTE                  int64         
 2   RISCHIO                     object        
 3   PRODOTTO                    object        
 4   STATO                       object        
 5   SIT ASSUN                   object        
 6   NUM_RATE                    int64         
 7   MODALITA_PAGAMENTO_STIPULA  object        
 8   RAGG_BUSINESS               object        
 9   AMT_PREMIO_RCA              float64       
 10  SCONTO_TOT_RCA              int64         
 11  AMT_PREMIO_CTTO             int64         
 12  AMT_SCONTO_CTTO             int64         
 13  GAR_BONUS_PROTECTION        int64         
 14  GAR_DANNI_CONTENUTO         int64         
 15  GAR_DANNI_FABBRICATO        int64         
 16  GAR_FURTO        

In [8]:
df_new['INCLUDES_1JAN2017'] = (df_new['DAT_DECORRENZA'] <= pd.Timestamp('2017-01-01')) & (df_new['DAT_CHIUSURA'] >= pd.Timestamp('2017-01-01'))
df_new['INCLUDES_1JAN2018'] = (df_new['DAT_DECORRENZA'] <= pd.Timestamp('2018-01-01')) & (df_new['DAT_CHIUSURA'] >= pd.Timestamp('2018-01-01'))
df_new['INCLUDES_1JUL2017'] = (df_new['DAT_DECORRENZA'] <= pd.Timestamp('2017-07-01')) & (df_new['DAT_CHIUSURA'] >= pd.Timestamp('2017-07-01'))
df_new['INCLUDES_1JUL2018'] = (df_new['DAT_DECORRENZA'] <= pd.Timestamp('2018-07-01')) & (df_new['DAT_CHIUSURA'] >= pd.Timestamp('2018-07-01'))
df_new['CONTRACT_LENGHT'] = (df_new['DAT_CHIUSURA'] - df_new['DAT_DECORRENZA']).dt.days
df_new

,CONTRACT_CODE,ID_CLIENTE,RISCHIO,PRODOTTO,STATO,SIT ASSUN,NUM_RATE,MODALITA_PAGAMENTO_STIPULA,RAGG_BUSINESS,AMT_PREMIO_RCA,...,GAR_RCOTHER,GAR_VANDALICI,DAT_DECORRENZA,DAT_CHIUSURA,DAT_SCADENZA,INCLUDES_1JAN2017,INCLUDES_1JAN2018,INCLUDES_1JUL2017,INCLUDES_1JUL2018,CONTRACT_LENGHT
1571600,0020454875&000106V09,1,Car,Auto,Sostituito,Riattivazione con modifica,0,Altro,DIRECT,0.00,...,0,0,2016-06-26,2016-10-01,2016-10-01,False,False,False,False,97
1571598,0020454875&000106V07,1,Car,Auto,Sostituito,Modifica,0,Carta di credito,DIRECT,41.20,...,0,0,2014-09-18,2014-11-16,2015-07-08,False,False,False,False,59
1571599,0020454875&000106V08,1,Car,Auto,Sostituito,Riattivazione,0,Altro,DIRECT,0.00,...,0,0,2015-05-30,2015-10-15,2016-01-19,False,False,False,False,138
1571602,0020454875&000110V11,1,Car,Auto,Sostituito,Riattivazione con modifica,0,Altro,DIRECT,0.00,...,0,0,2017-08-25,2017-10-08,2018-05-31,False,False,False,False,44
1571601,0020454875&000110,1,Car,Auto,Sostituito,Rinnovo,0,Carta di credito,DIRECT,256.34,...,0,0,2017-05-31,2017-08-23,2018-05-31,False,False,True,False,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1581356,0020462537&000109,1440378,Car,Auto,Sostituito,Rinnovo,0,Carta di credito,DIRECT,200.86,...,0,0,2016-07-05,2016-11-19,2017-07-05,False,False,False,False,137
1581355,0020462537&000106V08,1440378,Car,Auto,Sostituito,Riattivazione,0,Altro,DIRECT,0.00,...,0,0,2016-04-27,2016-07-01,2016-07-01,False,False,False,False,65
1581354,0020462537&000106V07,1440378,Car,Auto,Sostituito,Riattivazione,0,Altro,DIRECT,0.00,...,0,0,2015-06-09,2015-11-08,2016-01-12,False,False,False,False,152
1581357,0020462537&000109V10,1440378,Car,Auto,Sostituito,Riattivazione,0,Altro,DIRECT,0.00,...,0,0,2017-06-01,2017-11-03,2018-01-15,False,False,True,False,155


In [9]:
client = df_new[df_new["ID_CLIENTE"] == 1]
client

,CONTRACT_CODE,ID_CLIENTE,RISCHIO,PRODOTTO,STATO,SIT ASSUN,NUM_RATE,MODALITA_PAGAMENTO_STIPULA,RAGG_BUSINESS,AMT_PREMIO_RCA,...,GAR_RCOTHER,GAR_VANDALICI,DAT_DECORRENZA,DAT_CHIUSURA,DAT_SCADENZA,INCLUDES_1JAN2017,INCLUDES_1JAN2018,INCLUDES_1JUL2017,INCLUDES_1JUL2018,CONTRACT_LENGHT
1571600,0020454875&000106V09,1,Car,Auto,Sostituito,Riattivazione con modifica,0,Altro,DIRECT,0.00,...,0,0,2016-06-26,2016-10-01,2016-10-01,False,False,False,False,97
1571598,0020454875&000106V07,1,Car,Auto,Sostituito,Modifica,0,Carta di credito,DIRECT,41.20,...,0,0,2014-09-18,2014-11-16,2015-07-08,False,False,False,False,59
1571599,0020454875&000106V08,1,Car,Auto,Sostituito,Riattivazione,0,Altro,DIRECT,0.00,...,0,0,2015-05-30,2015-10-15,2016-01-19,False,False,False,False,138
1571602,0020454875&000110V11,1,Car,Auto,Sostituito,Riattivazione con modifica,0,Altro,DIRECT,0.00,...,0,0,2017-08-25,2017-10-08,2018-05-31,False,False,False,False,44
1571601,0020454875&000110,1,Car,Auto,Sostituito,Rinnovo,0,Carta di credito,DIRECT,256.34,...,0,0,2017-05-31,2017-08-23,2018-05-31,False,False,True,False,84
1571597,0020454875&000106,1,Car,Auto,Sostituito,Rinnovo,0,Carta di credito,DIRECT,259.02,...,0,0,2014-07-08,2014-09-18,2015-07-08,False,False,False,False,72
1571603,0020454875&000110V12,1,Car,Auto,Sostituito,Riattivazione,0,Altro,DIRECT,0.00,...,0,0,2018-06-04,2018-08-05,2019-01-25,False,False,False,True,62
1571605,0020454875&000110V14,1,Car,Auto,Sostituito,Riattivazione,0,Altro,DIRECT,0.00,...,0,0,2020-05-31,2020-08-16,2020-08-16,False,False,False,False,77
1571606,0020454875&000115,1,Car,Auto,Sospensione,Rinnovo,0,Carta di credito,DIRECT,257.05,...,0,0,2020-08-18,2020-10-18,2021-08-18,False,False,False,False,61
1571604,0020454875&000110V13,1,Car,Auto,Sostituito,Riattivazione con modifica,0,Altro,DIRECT,0.00,...,0,0,2019-07-11,2019-10-15,2019-12-31,False,False,False,False,96


In [10]:
mean_contract_length = df_new['CONTRACT_LENGHT'].mean()
mean_contract_length

275.645980225455

In [11]:
aggregated_df = df_new.groupby('ID_CLIENTE').agg({'INCLUDES_1JAN2017': 'any',
                                                   'INCLUDES_1JUL2017': 'any',
                                                   'INCLUDES_1JAN2018': 'any',
                                                   'INCLUDES_1JUL2018': 'any',
                                                   'CONTRACT_LENGHT': 'mean'})

In [12]:
aggregated_df['active_2017_2018'] = ((aggregated_df['INCLUDES_1JAN2017'] == True) & (aggregated_df['INCLUDES_1JAN2018'] == True)) | \
                                    ((aggregated_df['INCLUDES_1JUL2017'] == True) &
                                    (aggregated_df["INCLUDES_1JUL2018"] == True)).astype(int)
aggregated_df

,INCLUDES_1JAN2017,INCLUDES_1JUL2017,INCLUDES_1JAN2018,INCLUDES_1JUL2018,CONTRACT_LENGHT,active_2017_2018
ID_CLIENTE,,,,,,
1,False,True,False,True,79.000000,True
2,True,True,True,True,301.833333,True
3,False,False,False,False,365.000000,False
4,True,True,True,True,365.375000,True
5,True,True,True,False,365.250000,True
...,...,...,...,...,...,...
1440374,True,True,True,True,319.750000,True
1440375,False,False,False,False,68.666667,False
1440376,True,True,False,False,365.333333,False


In [13]:
aggregated_df.rename(columns={'CONTRACT_LENGHT': 'average_length'}, inplace=True)

In [14]:
aggregated_df

,INCLUDES_1JAN2017,INCLUDES_1JUL2017,INCLUDES_1JAN2018,INCLUDES_1JUL2018,average_length,active_2017_2018
ID_CLIENTE,,,,,,
1,False,True,False,True,79.000000,True
2,True,True,True,True,301.833333,True
3,False,False,False,False,365.000000,False
4,True,True,True,True,365.375000,True
5,True,True,True,False,365.250000,True
...,...,...,...,...,...,...
1440374,True,True,True,True,319.750000,True
1440375,False,False,False,False,68.666667,False
1440376,True,True,False,False,365.333333,False


In [15]:
counts_17_18 = aggregated_df["active_2017_2018"].value_counts()
counts_17_18

active_2017_2018
False    858031
True     543977
Name: count, dtype: int64

I think we should use 2016 and 2019 since the difference in the imbalancing it's not that much and 2019 will give us better results

In [16]:
aggregated_df["target"] = aggregated_df['active_2017_2018'].astype(int)
aggregated_df["target"] = aggregated_df["target"].map({0: 1, 1: 0})
aggregated_df

,INCLUDES_1JAN2017,INCLUDES_1JUL2017,INCLUDES_1JAN2018,INCLUDES_1JUL2018,average_length,active_2017_2018,target
ID_CLIENTE,,,,,,,
1,False,True,False,True,79.000000,True,0
2,True,True,True,True,301.833333,True,0
3,False,False,False,False,365.000000,False,1
4,True,True,True,True,365.375000,True,0
5,True,True,True,False,365.250000,True,0
...,...,...,...,...,...,...,...
1440374,True,True,True,True,319.750000,True,0
1440375,False,False,False,False,68.666667,False,1
1440376,True,True,False,False,365.333333,False,1


In [17]:
target = aggregated_df.drop(["INCLUDES_1JAN2017","INCLUDES_1JUL2017","INCLUDES_1JAN2018","INCLUDES_1JUL2018","average_length","active_2017_2018"], axis=1)
target

,target
ID_CLIENTE,
1,0
2,0
3,1
4,0
5,0
...,...
1440374,0
1440375,1
1440376,1


In [18]:
dfClientSort=dfClient.sort_values('ID_CLIENTE')
#Deleting columns with more than 10% of missing values
delete_columns = [colonna for colonna in dfClientSort.columns if dfClientSort[colonna].isnull().mean() > 0.1]

dfClientSort = dfClientSort.drop(delete_columns, axis=1)


In [19]:
aggregated_df2=pd.merge(dfClientSort,target, on ='ID_CLIENTE',how='left')
aggregated_df2=aggregated_df2.dropna(subset=["target"])
aggregated_df2["target"].astype(int)

0          0
1          0
2          1
3          0
4          0
          ..
1440373    0
1440374    1
1440375    1
1440376    0
1440377    0
Name: target, Length: 1402008, dtype: int32

In [20]:
dfClientContract= pd.merge(df_new,aggregated_df2, on="ID_CLIENTE", how="left")

In [21]:
dfClientContract["target"]=dfClientContract["target"].astype(int)
dfClientContract


,CONTRACT_CODE,ID_CLIENTE,RISCHIO,PRODOTTO,STATO,SIT ASSUN,NUM_RATE,MODALITA_PAGAMENTO_STIPULA,RAGG_BUSINESS,AMT_PREMIO_RCA,...,INCLUDES_1JUL2017,INCLUDES_1JUL2018,CONTRACT_LENGHT,COD_TIPO_PERSONA,RATING,COD_GEND_PF,FLG_APP,SUM_IMP_PREMIO_CTTO,ANNO_NASC_PF,target
0,0020454875&000106V09,1,Car,Auto,Sostituito,Riattivazione con modifica,0,Altro,DIRECT,0.00,...,False,False,97,PF,5.0,M,Y,1502.0,1977.0,0
1,0020454875&000106V07,1,Car,Auto,Sostituito,Modifica,0,Carta di credito,DIRECT,41.20,...,False,False,59,PF,5.0,M,Y,1502.0,1977.0,0
2,0020454875&000106V08,1,Car,Auto,Sostituito,Riattivazione,0,Altro,DIRECT,0.00,...,False,False,138,PF,5.0,M,Y,1502.0,1977.0,0
3,0020454875&000110V11,1,Car,Auto,Sostituito,Riattivazione con modifica,0,Altro,DIRECT,0.00,...,False,False,44,PF,5.0,M,Y,1502.0,1977.0,0
4,0020454875&000110,1,Car,Auto,Sostituito,Rinnovo,0,Carta di credito,DIRECT,256.34,...,True,False,84,PF,5.0,M,Y,1502.0,1977.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6748570,0020462537&000109,1440378,Car,Auto,Sostituito,Rinnovo,0,Carta di credito,DIRECT,200.86,...,False,False,137,PF,5.0,M,N,1017.0,1970.0,0
6748571,0020462537&000106V08,1440378,Car,Auto,Sostituito,Riattivazione,0,Altro,DIRECT,0.00,...,False,False,65,PF,5.0,M,N,1017.0,1970.0,0
6748572,0020462537&000106V07,1440378,Car,Auto,Sostituito,Riattivazione,0,Altro,DIRECT,0.00,...,False,False,152,PF,5.0,M,N,1017.0,1970.0,0
6748573,0020462537&000109V10,1440378,Car,Auto,Sostituito,Riattivazione,0,Altro,DIRECT,0.00,...,True,False,155,PF,5.0,M,N,1017.0,1970.0,0


In [22]:
#dfAuto = pd.read_csv( "CHURN_INFOAUTO.csv", encoding='ISO-8859-1')

tabelle_da_elimare = [colonna for colonna in dfInfoAuto.columns if dfInfoAuto[colonna].isnull().mean() > 0.2]

dfAuto2 = dfInfoAuto.drop(tabelle_da_elimare, axis=1)


In [23]:
#Final Dataset with all the tables merged
dataSet=pd.merge(dfClientContract,dfAuto2, on="CONTRACT_CODE",how="left")
dataSet = dataSet.drop(["INCLUDES_1JAN2017","INCLUDES_1JUL2017","INCLUDES_1JAN2018","INCLUDES_1JUL2018","CONTRACT_LENGHT"], axis=1)
#dataSet["target"]=dataSet["target"].astype(int)


In [24]:
dataSet.columns

Index(['CONTRACT_CODE', 'ID_CLIENTE', 'RISCHIO', 'PRODOTTO', 'STATO',
       'SIT ASSUN', 'NUM_RATE', 'MODALITA_PAGAMENTO_STIPULA', 'RAGG_BUSINESS',
       'AMT_PREMIO_RCA', 'SCONTO_TOT_RCA', 'AMT_PREMIO_CTTO',
       'AMT_SCONTO_CTTO', 'GAR_BONUS_PROTECTION', 'GAR_DANNI_CONTENUTO',
       'GAR_DANNI_FABBRICATO', 'GAR_FURTO', 'GAR_INC_ESPLOS_SCOPPIO',
       'GAR_INCFURTO', 'GAR_INFORTUNI_MALATTIA', 'GAR_KASKO_MINIKASKO',
       'GAR_RC_CAPOFAM', 'GAR_RC_PROPRIETA', 'GAR_RCA', 'GAR_RCOTHER',
       'GAR_VANDALICI', 'DAT_DECORRENZA', 'DAT_CHIUSURA', 'DAT_SCADENZA',
       'COD_TIPO_PERSONA', 'RATING', 'COD_GEND_PF', 'FLG_APP',
       'SUM_IMP_PREMIO_CTTO', 'ANNO_NASC_PF', 'target', 'COD_TIPO_VEICOLO',
       'DATA_PRIMA_IMMATRICOLAZIONE', 'COD_TIPOLOGIA_GUIDATORI',
       'KM_ANNUI_PREVISTI', 'COD_TIPO_ALIMENTAZIONE', 'DESC_MARCA'],
      dtype='object')

In [25]:
# dfContractAuto = pd.read_csv(os.path.join(
#     BASE_PATH, "CHURN_CONTRACT_AUTO.csv"))
# # Identifica le colonne che iniziano con "GAR_" e hanno più del XX% di valori nulli
# tabelle_da_elimare = [colonna for colonna in dfContractAuto.columns if dfContractAuto[colonna].isnull().mean() > 0.5]

# dfContractAuto = dfContractAuto.drop(tabelle_da_elimare, axis=1)
# na_status(dfContractAuto)

## Creating the target feature

To create the target variable we considered that any if the contract finished before its natural end, then it is a churn. We know this by subtracting the closing date (<i>dat_chiusura</i>) from the expiration data (<i>dat_scadenza</i>).

------------------------- CONTRACT_DATES ---------------------------------------------

Dataset containing information on relevant dates of the contracts

DAT_DECORRENZA: effective date (date since when the contract is effective)

DAT_CHIUSURA: closing date (when the contract has been closed by the client)

DAT_SCADENZA: expiration date (natural end of the contract)

In [26]:
#diff = df['DAT_SCADENZA'] - df['DAT_CHIUSURA']

As we transformet the dat_scadenza and the dat_chiusura in datetime type the diff column would have also a datetime type. So, in order to better anylise it we felt the need to add it in the dataframe as coolumn, but in a string form, in order to be more manipulable. During this analisis we saw that we couldn't create the target column using the diff list because one of the values was a '-' (one of the dat_chiusura was bigger than the dat_scadenza). What we did was to take out the only row that had this value in the diff column that we created. This cleaned our path to creat the target column.

In [27]:
#Creating the diff column in the dataframe
#df['diff'] = [str(x)[0] for x in diff ]

In [28]:
#seeing all the unique values of the diff column
#df['diff'].unique()

In [29]:
#Getting only the rows that didn't have the '-' value in the diff column (just one less row):
#df = df[df['diff'] != '-']

In [30]:
'''
Every time that the difference beteween the two columns (dat_chiusura & dat_scadenza) where bigger than 0
(when both finish at the same time) we would have a value 1 in the target column, else we would have a 0.
'''
#Creating the target column:
#df['target'] = [1 if int(str(x)[0]) > 0 else 0 for x in df['diff']]

'\nEvery time that the difference beteween the two columns (dat_chiusura & dat_scadenza) where bigger than 0\n(when both finish at the same time) we would have a value 1 in the target column, else we would have a 0.\n'

In [31]:
'''
As we have almost five million rows with 0 value in the target and 'just' 2 million rows with value 1 in the 
target column, we could do a little of undersampling, but we think that is better to not do anything right now because
we don't know the shape of the dataframe that we will use. If we will use all the data, with we will join, if we will 
with less sectors (this if there is more than one sector). We need first to explore our dataframe, our business, and our
objective before acting on this.
'''

#Is our target variable balanced?
#df['target'].value_counts()

"\nAs we have almost five million rows with 0 value in the target and 'just' 2 million rows with value 1 in the \ntarget column, we could do a little of undersampling, but we think that is better to not do anything right now because\nwe don't know the shape of the dataframe that we will use. If we will use all the data, with we will join, if we will \nwith less sectors (this if there is more than one sector). We need first to explore our dataframe, our business, and our\nobjective before acting on this.\n"

In [32]:
#df.columns

In [33]:
dataSet.head()

,CONTRACT_CODE,ID_CLIENTE,RISCHIO,PRODOTTO,STATO,SIT ASSUN,NUM_RATE,MODALITA_PAGAMENTO_STIPULA,RAGG_BUSINESS,AMT_PREMIO_RCA,...,FLG_APP,SUM_IMP_PREMIO_CTTO,ANNO_NASC_PF,target,COD_TIPO_VEICOLO,DATA_PRIMA_IMMATRICOLAZIONE,COD_TIPOLOGIA_GUIDATORI,KM_ANNUI_PREVISTI,COD_TIPO_ALIMENTAZIONE,DESC_MARCA
0,0020454875&000106V09,1,Car,Auto,Sostituito,Riattivazione con modifica,0,Altro,DIRECT,0.00,...,Y,1502.0,1977.0,0,31.0,01JAN2009:00:00:00,C,5000.0,NaN,Yamaha
1,0020454875&000106V07,1,Car,Auto,Sostituito,Modifica,0,Carta di credito,DIRECT,41.20,...,Y,1502.0,1977.0,0,31.0,01JAN2007:00:00:00,C,5000.0,NaN,Yamaha
2,0020454875&000106V08,1,Car,Auto,Sostituito,Riattivazione,0,Altro,DIRECT,0.00,...,Y,1502.0,1977.0,0,31.0,01JAN2007:00:00:00,C,5000.0,M,Yamaha
3,0020454875&000110V11,1,Car,Auto,Sostituito,Riattivazione con modifica,0,Altro,DIRECT,0.00,...,Y,1502.0,1977.0,0,31.0,01JAN2010:00:00:00,C,5000.0,NaN,Yamaha
4,0020454875&000110,1,Car,Auto,Sostituito,Rinnovo,0,Carta di credito,DIRECT,256.34,...,Y,1502.0,1977.0,0,31.0,01JAN2009:00:00:00,C,5000.0,NaN,Yamaha


In [34]:
# Doing the final aggreagation:
aggregations = {}
for column in dataSet.columns:
    aggregations[column] = 'last'

In [35]:
data = dataSet.groupby('ID_CLIENTE').agg(aggregations)

In [36]:
data.head()

,CONTRACT_CODE,ID_CLIENTE,RISCHIO,PRODOTTO,STATO,SIT ASSUN,NUM_RATE,MODALITA_PAGAMENTO_STIPULA,RAGG_BUSINESS,AMT_PREMIO_RCA,...,FLG_APP,SUM_IMP_PREMIO_CTTO,ANNO_NASC_PF,target,COD_TIPO_VEICOLO,DATA_PRIMA_IMMATRICOLAZIONE,COD_TIPOLOGIA_GUIDATORI,KM_ANNUI_PREVISTI,COD_TIPO_ALIMENTAZIONE,DESC_MARCA
ID_CLIENTE,,,,,,,,,,,,,,,,,,,,,
1,0020454875&000110V13,1,Car,Auto,Sostituito,Riattivazione con modifica,0,Altro,DIRECT,0.00,...,Y,1502.0,1977.0,0,31.0,01JAN2005:00:00:00,C,5000.0,M,Yamaha
2,0020461428&000205,2,Car,Auto,Sostituito,Rinnovo,1,Banca/Posta,DIRECT,456.06,...,Y,5031.0,1968.0,0,1.0,01JAN2006:00:00:00,C,5000.0,M,Lancia
3,0020460427&000104,3,Car,Auto,Chiuso (scaduto),Rinnovo,0,Carta di credito,DIRECT,387.85,...,N,2260.0,1969.0,1,1.0,01JAN2003:00:00:00,C,6000.0,G,Opel
4,0020458210&000103,4,Car,Auto,Sostituito,Rinnovo,0,Carta di credito,DIRECT,257.37,...,Y,4443.0,1958.0,0,1.0,01JAN2009:00:00:00,C,10000.0,G,Bmw
5,0020460908&000102,5,Car,Auto,Sostituito,Rinnovo,1,Carta di credito,DIRECT,355.04,...,N,4479.0,1961.0,0,1.0,01JAN2010:00:00:00,C,15000.0,G,Peugeot


## Final Data Exploration

In [37]:
data.shape

(1402008, 42)

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1402008 entries, 1 to 1440378
Data columns (total 42 columns):
 #   Column                       Non-Null Count    Dtype         
---  ------                       --------------    -----         
 0   CONTRACT_CODE                1402008 non-null  object        
 1   ID_CLIENTE                   1402008 non-null  int64         
 2   RISCHIO                      1402008 non-null  object        
 3   PRODOTTO                     1402008 non-null  object        
 4   STATO                        1402008 non-null  object        
 5   SIT ASSUN                    1402008 non-null  object        
 6   NUM_RATE                     1402008 non-null  int64         
 7   MODALITA_PAGAMENTO_STIPULA   1402008 non-null  object        
 8   RAGG_BUSINESS                1402008 non-null  object        
 9   AMT_PREMIO_RCA               1402008 non-null  float64       
 10  SCONTO_TOT_RCA               1402008 non-null  int64         
 11  AMT_PREMIO_CTTO 

In [39]:
data.describe()

,ID_CLIENTE,NUM_RATE,AMT_PREMIO_RCA,SCONTO_TOT_RCA,AMT_PREMIO_CTTO,AMT_SCONTO_CTTO,GAR_BONUS_PROTECTION,GAR_DANNI_CONTENUTO,GAR_DANNI_FABBRICATO,GAR_FURTO,...,GAR_VANDALICI,DAT_DECORRENZA,DAT_CHIUSURA,DAT_SCADENZA,RATING,SUM_IMP_PREMIO_CTTO,ANNO_NASC_PF,target,COD_TIPO_VEICOLO,KM_ANNUI_PREVISTI
count,1.402008e+06,1.402008e+06,1.402008e+06,1.402008e+06,1.402008e+06,1.402008e+06,1.402008e+06,1402008.0,1402008.0,1402008.0,...,1.402008e+06,1402008,1402008,1402008,1.401924e+06,1.402007e+06,1.355818e+06,1.402008e+06,1.399201e+06,1.391341e+06
mean,7.192054e+05,2.193668e-01,3.784200e+02,1.894482e+02,4.645774e+02,5.956296e+01,9.140604e-02,0.0,0.0,0.0,...,5.386631e-02,2017-04-11 13:15:16.130577920,2018-02-05 03:22:26.667333376,2018-03-14 20:35:12.412483072,5.098802e+00,3.972732e+03,1.967318e+03,6.120015e-01,9.616682e+00,8.056364e+03
min,1.000000e+00,0.000000e+00,-1.293900e+03,-8.290000e+02,-2.630000e+03,-8.700000e+02,0.000000e+00,0.0,0.0,0.0,...,0.000000e+00,2013-09-22 00:00:00,2013-11-27 00:00:00,2015-01-01 00:00:00,0.000000e+00,1.000000e+00,1.899000e+03,0.000000e+00,1.000000e+00,0.000000e+00
25%,3.576998e+05,0.000000e+00,2.202000e+02,3.700000e+01,2.720000e+02,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.000000e+00,2015-05-04 00:00:00,2016-01-29 00:00:00,2016-03-26 00:00:00,5.000000e+00,8.600000e+02,1.959000e+03,0.000000e+00,1.000000e+00,5.000000e+03
50%,7.179985e+05,0.000000e+00,3.377500e+02,1.400000e+02,4.290000e+02,5.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.000000e+00,2017-01-18 00:00:00,2017-10-28 00:00:00,2017-12-03 00:00:00,5.000000e+00,2.147000e+03,1.968000e+03,1.000000e+00,1.000000e+00,5.000000e+03
75%,1.080206e+06,0.000000e+00,5.009200e+02,2.590000e+02,6.270000e+02,6.500000e+01,0.000000e+00,0.0,0.0,0.0,...,0.000000e+00,2019-04-26 00:00:00,2020-02-18 00:00:00,2020-03-30 00:00:00,6.000000e+00,5.083000e+03,1.977000e+03,1.000000e+00,3.100000e+01,1.000000e+04
max,1.440378e+06,4.000000e+00,5.675190e+03,5.089000e+03,5.818000e+03,4.770000e+03,1.000000e+00,0.0,0.0,0.0,...,1.000000e+00,2021-04-01 00:00:00,2022-03-19 00:00:00,2022-03-19 00:00:00,9.000000e+00,1.387536e+07,2.001000e+03,1.000000e+00,7.500000e+01,1.000000e+06
std,4.165698e+05,5.447393e-01,2.703279e+02,2.126212e+02,3.176617e+02,1.174293e+02,2.881858e-01,0.0,0.0,0.0,...,2.257538e-01,NaN,NaN,NaN,9.357952e-01,1.280640e+04,1.345584e+01,4.872944e-01,1.359363e+01,7.293894e+03


In [40]:
#data.hist();

In [41]:
data.columns

Index(['CONTRACT_CODE', 'ID_CLIENTE', 'RISCHIO', 'PRODOTTO', 'STATO',
       'SIT ASSUN', 'NUM_RATE', 'MODALITA_PAGAMENTO_STIPULA', 'RAGG_BUSINESS',
       'AMT_PREMIO_RCA', 'SCONTO_TOT_RCA', 'AMT_PREMIO_CTTO',
       'AMT_SCONTO_CTTO', 'GAR_BONUS_PROTECTION', 'GAR_DANNI_CONTENUTO',
       'GAR_DANNI_FABBRICATO', 'GAR_FURTO', 'GAR_INC_ESPLOS_SCOPPIO',
       'GAR_INCFURTO', 'GAR_INFORTUNI_MALATTIA', 'GAR_KASKO_MINIKASKO',
       'GAR_RC_CAPOFAM', 'GAR_RC_PROPRIETA', 'GAR_RCA', 'GAR_RCOTHER',
       'GAR_VANDALICI', 'DAT_DECORRENZA', 'DAT_CHIUSURA', 'DAT_SCADENZA',
       'COD_TIPO_PERSONA', 'RATING', 'COD_GEND_PF', 'FLG_APP',
       'SUM_IMP_PREMIO_CTTO', 'ANNO_NASC_PF', 'target', 'COD_TIPO_VEICOLO',
       'DATA_PRIMA_IMMATRICOLAZIONE', 'COD_TIPOLOGIA_GUIDATORI',
       'KM_ANNUI_PREVISTI', 'COD_TIPO_ALIMENTAZIONE', 'DESC_MARCA'],
      dtype='object')

In [42]:
correlation = data[['RATING', 'target', 'NUM_RATE', 'AMT_PREMIO_RCA', 'SCONTO_TOT_RCA', 'AMT_PREMIO_CTTO', 
                   'AMT_SCONTO_CTTO', 'GAR_BONUS_PROTECTION', 'GAR_INCFURTO', 'GAR_INFORTUNI_MALATTIA', 
                  'GAR_KASKO_MINIKASKO', 'ANNO_NASC_PF', 'COD_TIPO_VEICOLO']].corr()

In [43]:
#correlation_heatmap(correlation)

### Feature Selection

In [44]:
# We will drop the datetime columns because they were used to get to our target column
data.drop(['DAT_DECORRENZA', 'DAT_CHIUSURA', 'DAT_SCADENZA'], axis=1, inplace=True)

In [45]:
# We will delete this columns because they have just one value which means that they do not add information
data.drop(['GAR_DANNI_FABBRICATO', 'GAR_DANNI_FABBRICATO', 'GAR_FURTO', 'GAR_INC_ESPLOS_SCOPPIO', 
          'GAR_RC_CAPOFAM', 'GAR_RC_PROPRIETA', 'GAR_RCA', 'GAR_RCOTHER', 'KM_ANNUI_PREVISTI'], axis=1, inplace=True)

In [46]:
data.shape

(1402008, 31)

### Preprocessing

#### Dealing with the NAs

In [47]:
data.isnull().sum()

CONTRACT_CODE                      0
ID_CLIENTE                         0
RISCHIO                            0
PRODOTTO                           0
STATO                              0
SIT ASSUN                          0
NUM_RATE                           0
MODALITA_PAGAMENTO_STIPULA         0
RAGG_BUSINESS                      0
AMT_PREMIO_RCA                     0
SCONTO_TOT_RCA                     0
AMT_PREMIO_CTTO                    0
AMT_SCONTO_CTTO                    0
GAR_BONUS_PROTECTION               0
GAR_DANNI_CONTENUTO                0
GAR_INCFURTO                       0
GAR_INFORTUNI_MALATTIA             0
GAR_KASKO_MINIKASKO                0
GAR_VANDALICI                      0
COD_TIPO_PERSONA                   0
RATING                            84
COD_GEND_PF                    46190
FLG_APP                            0
SUM_IMP_PREMIO_CTTO                1
ANNO_NASC_PF                   46190
target                             0
COD_TIPO_VEICOLO                2807
D

For the COD_GEND_PF and ANNO_NASC_PF we cannot drop the rows because if we did we would get rid of all the information concerning the companies as clients. So for the first we will use 'C' to fill the NAs and for the other we will use the mean. For the others as they are few regarding the whole size of the dataset we will drop the rows containing the NAs.

In [48]:
data['COD_GEND_PF'].fillna('C', inplace=True)
data['ANNO_NASC_PF'].fillna(data['ANNO_NASC_PF'].mean(), inplace=True)
data['RATING'].fillna(data['RATING'].mean(), inplace=True)

In [49]:
to_drop = ['SUM_IMP_PREMIO_CTTO', 'COD_TIPO_VEICOLO', 'DATA_PRIMA_IMMATRICOLAZIONE', 
           'COD_TIPOLOGIA_GUIDATORI', 'COD_TIPO_ALIMENTAZIONE', 'DESC_MARCA']

In [50]:
for column in to_drop:
    data.dropna(subset=column, inplace=True)

In [51]:
data.isna().sum()

CONTRACT_CODE                  0
ID_CLIENTE                     0
RISCHIO                        0
PRODOTTO                       0
STATO                          0
SIT ASSUN                      0
NUM_RATE                       0
MODALITA_PAGAMENTO_STIPULA     0
RAGG_BUSINESS                  0
AMT_PREMIO_RCA                 0
SCONTO_TOT_RCA                 0
AMT_PREMIO_CTTO                0
AMT_SCONTO_CTTO                0
GAR_BONUS_PROTECTION           0
GAR_DANNI_CONTENUTO            0
GAR_INCFURTO                   0
GAR_INFORTUNI_MALATTIA         0
GAR_KASKO_MINIKASKO            0
GAR_VANDALICI                  0
COD_TIPO_PERSONA               0
RATING                         0
COD_GEND_PF                    0
FLG_APP                        0
SUM_IMP_PREMIO_CTTO            0
ANNO_NASC_PF                   0
target                         0
COD_TIPO_VEICOLO               0
DATA_PRIMA_IMMATRICOLAZIONE    0
COD_TIPOLOGIA_GUIDATORI        0
COD_TIPO_ALIMENTAZIONE         0
DESC_MARCA

### Scalling 

In [52]:
# ---------------------------------- Scalling the categoricals ------------------------------------

# Creating the encoder:
cat_encoder = CatBoostEncoder()

# Creating the list with the categorical columns:
cat_col = ['CONTRACT_CODE', 'RISCHIO', 'PRODOTTO', 'STATO', 'SIT ASSUN', 'MODALITA_PAGAMENTO_STIPULA', 'RAGG_BUSINESS',
          'COD_TIPO_PERSONA', 'COD_GEND_PF', 'FLG_APP', 'DATA_PRIMA_IMMATRICOLAZIONE', 'COD_TIPOLOGIA_GUIDATORI', 
          'COD_TIPO_ALIMENTAZIONE', 'DESC_MARCA']

# Fit and transform the categorical columns
df_encoded = cat_encoder.fit_transform(data[cat_col], data['target'])

# Concatenate the encoded categorical columns with the original DataFrame
data = pd.concat([data.drop(columns=cat_col, axis=1), df_encoded], axis=1)

In [53]:
# ---------------------------------- Scalling the numericals -----------------------------------------
# Creating the scaler:
scaler = MinMaxScaler()

numericals = ['ID_CLIENTE', 'NUM_RATE', 'AMT_PREMIO_RCA', 'SCONTO_TOT_RCA',
       'AMT_PREMIO_CTTO', 'AMT_SCONTO_CTTO', 'GAR_BONUS_PROTECTION',
       'GAR_DANNI_CONTENUTO', 'GAR_INCFURTO', 'GAR_INFORTUNI_MALATTIA',
       'GAR_KASKO_MINIKASKO', 'GAR_VANDALICI', 'RATING', 'SUM_IMP_PREMIO_CTTO',
       'ANNO_NASC_PF', 'COD_TIPO_VEICOLO']

data[numericals] = scaler.fit_transform(data[numericals])

### Dividing the dataset into X, y and train, val, test

In [54]:
X = data.drop('target', axis=1)
y = data['target']

In [55]:
# Spliting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2,
                                                    random_state=42)

# Spliting the train set into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                  y_train,
                                                  test_size=0.3,
                                                  random_state=42)


### Modeling

#### Logistic Regression

In [60]:
#Creating and fitting the Logistic Regression model:
lr = LogisticRegression(random_state=rs, max_iter=10000)
lr.fit(X_train, y_train);

#Predicting:
y_lr = lr.predict(X_val)

#Seeing the accuracy:
accuracy_lr = accuracy_score(y_val, y_lr) * 100

# Other indicators:
precision_lr = precision_score(y_val, y_lr, average='macro') * 100
recall_lr = recall_score(y_val, y_lr, average='macro') * 100
f1_lr = f1_score(y_val, y_lr, average='macro') * 100



print("The accuracy on validation set is {0:.2f}%".format(accuracy_lr))
print("The Recall on validation set is {0:.2f}%".format(recall_lr))
print("The Precision on validation set is {0:.2f}%".format(precision_lr))
print("The F1 Score on validation set is {0:.2f}%".format(f1_lr))

The accuracy on validation set is 78.72%
The Recall on validation set is 77.60%
The Precision on validation set is 77.97%
The F1 Score on validation set is 77.77%


#### XGBClassifier

In [61]:
#Creating and fitting the XGBoost model:
xgboost = XGBClassifier(random_state=rs )
xgboost.fit(X_train, y_train);

#Predicting:
y_xgboost = xgboost.predict(X_val)

#Seeing the accuracy:
accuracy_xgb = accuracy_score(y_val, y_xgboost) * 100

# Other indicators:
precision_xgb = precision_score(y_val, y_xgboost, average='macro') * 100
recall_xgb = recall_score(y_val, y_xgboost, average='macro') * 100
f1_xgb = f1_score(y_val, y_xgboost, average='macro') * 100



print("The accuracy on validation set is {0:.2f}%".format(accuracy_xgb))
print("The Recall on validation set is {0:.2f}%".format(recall_xgb))
print("The Precision on validation set is {0:.2f}%".format(precision_xgb))
print("The F1 Score on validation set is {0:.2f}%".format(f1_xgb))

The accuracy on validation set is 85.94%
The Recall on validation set is 85.58%
The Precision on validation set is 85.34%
The F1 Score on validation set is 85.45%


#### Random Forest

In [62]:
#Creating and fitting the Random Forest model:
rf = RandomForestClassifier(random_state=rs)
rf.fit(X_train, y_train);

#Predicting:
y_rf = rf.predict(X_val)

#Seeing the accuracy:
accuracy_rf = accuracy_score(y_val, y_rf) * 100

# Other indicators:
precision_rf = precision_score(y_val, y_rf, average='macro') * 100
recall_rf = recall_score(y_val, y_rf, average='macro') * 100
f1_rf = f1_score(y_val, y_rf, average='macro') * 100



print("The accuracy on validation set is {0:.2f}%".format(accuracy_rf))
print("The Recall on validation set is {0:.2f}%".format(recall_rf))
print("The Precision on validation set is {0:.2f}%".format(precision_rf))
print("The F1 Score on validation set is {0:.2f}%".format(f1_rf))

The accuracy on validation set is 85.74%
The Recall on validation set is 85.38%
The Precision on validation set is 85.13%
The F1 Score on validation set is 85.25%


#### Neural Network

In [64]:
# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Making predictions on the test data
y_pred = model.predict(X_test)
y_pred_binary = np.round(y_pred).flatten()

# Calculating precision, recall, and F1 score
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Epoch 1/10
22759/22759 [==============================] - 91s 4ms/step - loss: 0.4174 - accuracy: 0.7998 - val_loss: 0.4000 - val_accuracy: 0.8049
Epoch 2/10
22759/22759 [==============================] - 87s 4ms/step - loss: 0.3704 - accuracy: 0.8277 - val_loss: 0.3842 - val_accuracy: 0.8173
Epoch 3/10
22759/22759 [==============================] - 86s 4ms/step - loss: 0.3593 - accuracy: 0.8336 - val_loss: 0.3460 - val_accuracy: 0.8408
Epoch 4/10
22759/22759 [==============================] - 86s 4ms/step - loss: 0.3526 - accuracy: 0.8373 - val_loss: 0.3456 - val_accuracy: 0.8403
Epoch 5/10
22759/22759 [==============================] - 113s 5ms/step - loss: 0.3486 - accuracy: 0.8395 - val_loss: 0.3370 - val_accuracy: 0.8459
Epoch 6/10
22759/22759 [==============================] - 85s 4ms/step - loss: 0.3452 - accuracy: 0.8411 - val_loss: 0.3398 - val_accuracy: 0.8442
Epoch 7/10
22759/22759 [==============================] - 87s 4ms/step - loss: 0.3424 - accuracy: 0.8425 - val_loss: 